## Evolutionary Algorithms Demo

The data for this demo is the same as seen previously (i.e., sexual clinic location problem).

### Imports 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# you will use itertools for enumerating all solutions in small instances.
from itertools import combinations

###  `metapy` package imports

The package contains the classes and functions for the evolutionary algorithms you will use in this notebook.

In [ ]:
from metapy.evolutionary.evolutionary import (EvolutionaryAlgorithm, 
                                              MuLambdaEvolutionStrategy, 
                                              MuPlusLambdaEvolutionStrategy,
                                              GeneticAlgorithmStrategy,
                                              ElitistGeneticAlgorithmStrategy,
                                              WeightedAverageObjective,
                                              FacilityLocationPopulationGenerator,
                                              BasicFacilityLocationMutator,
                                              TournamentSelector,
                                              FacilityLocationSinglePointCrossOver)

Function to determine the number of possible combinations...

In [ ]:
def combination_counter(n_facilities, p):
    facility = np.arange(n_facilities, dtype=np.uint8)
    combs = [np.array(a) for a in combinations(facility, p)]
    combs_len = len(combs)
    combs_len = format(combs_len, ",")
    print(f"Placing {p} facilities from a possible {n_facilities} " +
          f"location yields {combs_len} possible combinations")

Other functions and data storage (dict) required.

In [ ]:
def random_restarts(max_iter, obj, n_facilities, p, random_seed=None):
    '''
    max_iter : int
        Maximum number of iterations to try
    
    obj :  object
        WeightedAverageObjective
    
    n_facilities : int
        The number of candidate locations where you could place facilities (clinics).
        
    p : int
        The number of clinics to place.
    
    Returns
    -------
    
    best_cost : float
        Lowest 'cost'    
    
    solution : array
        Indecies of clincs in solution.
    '''
    
    np.random.seed(random_seed)
    
    # implementation of random restarts alg
    best_cost = np.Inf
    best_solution = None
    for i in range(max_iter):
        solution = random_solution(n_facilities, p)
        cost = obj.evaluate(solution)
        
        if cost < best_cost:
            best_cost = cost
            best_solution = solution
            
    return best_cost, solution
        

In [ ]:
def random_solution(n_candidates, p, random_seed=None):
    '''
    Helper function to generate a random solution
    
    Params
    ------
    n_candidates : int
        The number of candidate locations where you could place 
        clinics (facilities).
        
    p : int
        The number of clinics to place.
        
    random_seed : int (Default=None)
        Random seed for reproducibility.
    
    Returns
    -------
    
    Vector (np.array) of length p
    '''
    # create a random number generator
    rng = np.random.default_rng(seed=random_seed)

    # sample without replacement
    solution = []
    while len(solution) < p:
        candidate = rng.integers(0, n_candidates)
        if candidate not in solution:
            solution.append(candidate)
            
    return np.array(solution)

In [ ]:
# create a dictionary to store results
_results = {}

### Import case study data

The car travel times in minutes from annoymised postcode sectors to annoymised clinic locations.

In [ ]:
travel_matrix = pd.read_csv('../data/clinic_car_travel_time.csv', 
                            index_col='sector')
travel_matrix.head()

In [ ]:
# no of cases by postcode sector...

cases = pd.read_csv('../data/sh_demand.csv', index_col='sector')
cases.head()

#### 1. Generating an initial population.

The first task when using a population based method is to create an initial random population of solutions!  For our purposes, this is a multi-dimensional array.  We can use an object of type `FacilityLocationPopulationGenerator` to do the work for us here.

```python
from metapy.evolutionary.evolutionary import FacilityLocationPopulationGenerator
```

`FacilityLocationPopulationGenerator` accepts three arguments when it is created:

* `n_candidates`: int.  This is $P$ the number of candidate locations
* `n_facilities`: int. This is $p$ the number of facilities to place.
* `random_seed`: int, optional (default=None).  Set if you want a reproducible result.  For example = 42.

`FacilityLocationPopulationGenerator` has a single method `generate` that accepts a parameter specifying the population size.  It returns a multi-dimensional numpy array.

Let's assume you want have a problem with $P$ = 28, $p$ = 8 and we want to create a population of size 10.

```python
#example solution
N_CANDIDATES = 28
N_FACILITIES = 8
SEED = 42
POPULATION_SIZE = 10

gen = FacilityLocationPopulationGenerator(n_candidates=N_CANDIDATES,
                                          n_facilities=N_FACILITIES,
                                          random_seed=SEED)


gen.generate(population_size=POPULATION_SIZE)
```

In [ ]:
combination_counter(28, 8)

In [ ]:
# example solution
N_CANDIDATES = 28
N_FACILITIES = 8
SEED = 42
POPULATION_SIZE = 10

# Instantiate the FacilityLocationPopulationGenerator class into an object variable (gen)
gen = FacilityLocationPopulationGenerator(n_candidates=N_CANDIDATES,
                                          n_facilities=N_FACILITIES,
                                          random_seed=SEED)

# Remember the 'gen' object as a single method generate 
# that accepts a parameter specifying the population size. 
gen.generate(population_size=10)

#### 2: Mutating a solution

* Basic evolutionary strategies work by mutating the most promising solutions in the population.  
* There are many ways to implement mutation.  
* Here you will use `BasicFacilityLocationMutator`.  
* Each element in a solution has a constant probability of mutation (by default 1 / no. of facilities in a solution, but you may wish to set this higher.).  
* If a facility is chosen then it is replaced by a random facility not currently in the solution.

You can create a `BasicFacilityLocationMutator` as follows:

```python
mutator = BasicFacilityLocationMutator(n_candidates=28,
                                       solution_size=4)
solution = np.array([1, 2, 3, 4])

mutant = mutator.mutate(solution)
print(mutant)
```

To use a higher mutation rate:

```python
mutator = BasicFacilityLocationMutator(n_candidates=28,
                                  solution_size=4,
                                  mutation_rate=0.6)
solution = np.array([1, 2, 3, 4])

mutant = mutator.mutate(solution)
print(mutant)
```

In [ ]:
# mutating through 10 generations
gens = 10


mutator = BasicFacilityLocationMutator(n_candidates=28,
                                       solution_size=4,
                                       mutation_rate=0.5)
solution = np.arange(5)

for i in range(gens):
    solution = mutator.mutate(solution)
    print(f'solution {sorted(solution)}')

#### Exercise 3: The <span style="color:red"> $(\mu, \lambda)$ </span> and <span style="color:blue">$(\mu+\lambda)$ </span> evolutionary strategies

A random initial population and a mutation operator provide the ingredients for the two basic evolutionary strategies: <span style="color:red"> $(\mu, \lambda)$ </span> and <span style="color:blue">$(\mu+\lambda)$ </span>.

Remember...
* $(\lambda)$ "lambda" represents initial population size (i.e. how many solutions)
* $(\mu)$ "mu" represents the number of best solutions to keep, remainder are deleted

Below you will :
* Determine how many possible combinations there are to consider using the `combination_counter` function defined above.
* Run two evolutionary algorithms with <span style="color:red">$(\mu, \lambda)$</span> and <span style="color:blue">$(\mu$ **+** $\lambda)$</span>  strategies respectively.
* Investigate the parameters required when creating <span style="color:red">**MuLambdaEvolutionStrategy**</span>  and <span style="color:blue"> **MuPlusLambdaEvolutionStrategy** </span> 
* Use a problem size of 28 candidate locations and 14 facilities
* Initially try 
 * $\mu $("mu" aka number of best solutions to keep) = 10; and 
 * $\lambda$ ("lambda" aka initial population size) = 200. 
* Using a random initial population evolve for 50 generations.
* Compare with random restarts (from code along file).

**Note**:
* Evolutionary strategies are computationally expensive.  Expect a 50 generation algorithm to take 20-45 seconds on your machine.

Read about `%%time` [here](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-time)

In [ ]:
combination_counter(28, 14)

In [ ]:
%%time

# Evolutionary Algorithm - (mu,lambda) strategy
n_candidates = 28
n_facilities = 14

mu = 10
_lambda = 200

# objective
objective = WeightedAverageObjective(cases, travel_matrix)

# initial solution generator
init = FacilityLocationPopulationGenerator(n_candidates, n_facilities)

# mutation operator
mutator = BasicFacilityLocationMutator(n_candidates=n_candidates, 
                                       solution_size=n_facilities, 
                                       mutation_rate=0.2, verbose=False)

# evolutionary stategy - mu, lambda
strategy = MuLambdaEvolutionStrategy(mu, _lambda, mutator)

# solver object
solver = EvolutionaryAlgorithm(init, objective, _lambda, strategy, 
                               maximisation=False, generations=50)

print("\nRunning (mu, lambda) evolutionary alg...")
solver.solve()

print("\n** (MU,LAMBDA) OUTPUT ***")
print("best cost:\t{0}".format(solver.best_fitness))
print("best solutions:")
print(sorted(solver.best_solution))
print("\n ------")

_results['mulambda'] = solver.best_fitness

In [ ]:
%%time

# Evolutionary Algorithm - (mu+lambda) strategy
n_candidates = 28
n_facilities = 14

mu = 10
_lambda = 200

# objective
objective = WeightedAverageObjective(cases, travel_matrix)

# initial solution generator
init = FacilityLocationPopulationGenerator(n_candidates, n_facilities)

# mutation operator
mutator = BasicFacilityLocationMutator(n_candidates=n_candidates, 
                                       solution_size=n_facilities, 
                                       mutation_rate=0.2, verbose=False)

# evolutionary stategy - mu PLUS lambda
strategy = MuPlusLambdaEvolutionStrategy(mu, _lambda, mutator)

# solver object
solver = EvolutionaryAlgorithm(init, objective, _lambda, strategy, 
                               maximisation=False, generations=50)

print("\nRunning (mu + lambda) evolutionary alg...")
solver.solve()

print("\n** (MU+LAMBDA) OUTPUT ***")
print("best cost:\t{0}".format(solver.best_fitness))
print("best solutions:")
print(sorted(solver.best_solution))
print("\n ------")

_results['mupluslamba'] = solver.best_fitness

In [ ]:
%%time

rr_bc, rr_bs = random_restarts(10000,
                               objective,
                               28, 
                               14)

print(rr_bc)
print(rr_bs)

_results['random_restarts'] = rr_bc

#### Exercise 4: Locating facilities using a full Genetic Algorithm (GA)

Now that we have warmed up using  $(\mu, \lambda)$ and $(\mu+\lambda)$  it is time to move onto a full GA.  This means you need to take account of two further steps.

* A selection operator for breeding - in this instance you will use the provided `TournamentSelector`
* A crossover operator for breeding - you will use `FacilityLocationSinglePointCrossover`

See lecture slides for details of how these work.

`metapy` provides standard and elitist GA strategies.  The code provided below demonstrates how these are instantiated and used to solve the sexual health clinic facility location problem.


Below you will :
* Run `GeneticAlgorithmStrategy` and `ElitistGeneticAlgorithmStrategy` using the parameters provided.
* Compare the results of all above.

In [ ]:
%%time

# Evolutionary Algorithm - Genetic Algorithm strategy

n_candidates = 28
n_facilities = 14

_lambda = 200 

# objective
objective = WeightedAverageObjective(cases, travel_matrix)

# initial solution generator
init = FacilityLocationPopulationGenerator(n_candidates, n_facilities)

# mutation operator
mutator = BasicFacilityLocationMutator(n_candidates=n_candidates, 
                                       solution_size=n_facilities, 
                                       mutation_rate=0.2, verbose=False)

# cross over operator
x_over = FacilityLocationSinglePointCrossOver()

#GA strategy
strategy = GeneticAlgorithmStrategy(_lambda, 
                                    selector=TournamentSelector(),
                                    xoperator=x_over,
                                    mutator=mutator)


solver = EvolutionaryAlgorithm(init, objective,_lambda, strategy, 
                               maximisation=False, generations=50)
print("\nRunning Genetic Algorithm")
solver.solve()

print("\n** GA OUTPUT ***")
print("best cost:\t{0}".format(solver.best_fitness))
print("best solutions:")
print(solver.best_solution)

_results['ga'] = solver.best_fitness

In [ ]:
%%time 

# Evolutionary Algorithm - Elistist Genetic Algorithm strategy

n_candidates = 28
n_facilities = 14

# GA parameters
mu = 10 
_lambda = 200

# objective
objective = WeightedAverageObjective(cases, travel_matrix)

# initial solution generator
init = FacilityLocationPopulationGenerator(n_candidates, n_facilities)

# mutation operator
mutator = BasicFacilityLocationMutator(n_candidates=n_candidates, 
                                       solution_size=n_facilities, 
                                       mutation_rate=0.2, verbose=False)

# cross over operator
x_over = FacilityLocationSinglePointCrossOver()

# GA strategy
strategy = ElitistGeneticAlgorithmStrategy(mu,
                                           _lambda, 
                                           selector=TournamentSelector(),
                                           xoperator=x_over,
                                           mutator=mutator)


solver = EvolutionaryAlgorithm(init,
                               objective,
                               _lambda,
                               strategy, 
                               maximisation=False, generations=50)
print("\nRunning Elitist Genetic Algorithm")
solver.solve()

print("\n** ELITIST GA OUTPUT ***")
print("best cost:\t{0}".format(solver.best_fitness))
print("best solutions:")
print(solver.best_solution)

_results['ga_elitism'] = solver.best_fitness

Comparing results....

In [ ]:
_results

In [ ]:
optimal_index = np.argmin(_results)

# Create axis on which to generate the chart
fig, ax = plt.subplots(1, 1, figsize=(8, 5))

barlist = ax.bar(_results.keys(), _results.values())
optimal_index = np.argmin(_results.values())
barlist[optimal_index].set_color('r')


# End